In [2]:
import pandas as pd
import numpy as np

# Load R_DrugBank.txt
DTI_table = pd.read_csv("R_DrugBank.txt", delimiter="\t", header = None)
DTI_table.columns = ["Target_ID", "Drug_ID"]
DTI_table[DTI_table["Drug_ID"] == "DB00672"]

,Target_ID,Drug_ID
0,Q16348,DB00672
539,P23219,DB00672
4527,P46059,DB00672
4970,P08183,DB00672
5222,Q09428,DB00672
5249,Q4U2R8,DB00672
5543,P11712,DB00672
5819,P33261,DB00672


In [3]:
# load smiles information table
smiles = pd.read_csv("dti2vec_drugbankid2smiles", delimiter="\t", header = None)
smiles.columns = ["Drug_ID", "Drug"]
DTI_table_1 = pd.merge(DTI_table, smiles, on = "Drug_ID").drop_duplicates()

In [5]:
# load sequence information table
seq = pd.read_csv("origin_proteins.csv").drop("Unnamed: 0",axis = 1)
seq.columns = ["Target_ID", "Target"]
DTI_table_2 = pd.merge(DTI_table_1, seq, on = "Target_ID").drop_duplicates()

In [144]:
#concat sequence, smiles, target, drug, relation

DTI_table_2

,Target_ID,Drug_ID,Drug,Target,rel
0,Q16348,DB00672,CCCNC(=O)NS(=O)(=O)C1=CC=C(C=C1)Cl,MNPFQKNESKETLFSPVSIEEVPPRPPSPPKKPSPTICGSNYPLSI...,1
1,Q16348,DB00577,CC(C)[C@@H](C(=O)OCCOCN1C=NC2=C1N=C(NC2=O)N)N,MNPFQKNESKETLFSPVSIEEVPPRPPSPPKKPSPTICGSNYPLSI...,1
2,Q16348,DB01212,CN1C(=NC(=O)C(=O)N1)SCC2=C(N3[C@@H]([C@@H](C3=...,MNPFQKNESKETLFSPVSIEEVPPRPPSPPKKPSPTICGSNYPLSI...,1
3,Q16348,DB01112,CO/N=C(/C1=CC=CO1)\C(=O)N[C@H]2[C@@H]3N(C2=O)C...,MNPFQKNESKETLFSPVSIEEVPPRPPSPPKKPSPTICGSNYPLSI...,1
4,Q16348,DB00456,CC(=O)OCC1=C(N2[C@@H]([C@@H](C2=O)NC(=O)CC3=CC...,MNPFQKNESKETLFSPVSIEEVPPRPPSPPKKPSPTICGSNYPLSI...,1
...,...,...,...,...,...
9859,P25815,DB01003,C1=CC2=C(C(=C1)OCC(COC3=CC=CC4=C3C(=O)C=C(O4)C...,MTELETAMGMIIDVFSRYSGSEGSTQTLTKGELKVLMEKELPGFLQ...,1
9860,P09110,DB09069,COC1=C(C(=C(C=C1)CN2CCNCC2)OC)OC,MQRLQVVLGHLRGPADSGWMPQAAPCLSGAPQASAADVVVVHGRRT...,1
9861,P12314,DB00992,COC(=O)CCC(=O)CN,MWFLTTLLLWVPVDGQVDTTKAVITLQPPWVSVFQEETVTLHCEVL...,1
9862,P25092,DB08890,C[C@H]1C(=O)N[C@H]2CSSC[C@H]3C(=O)N[C@H](C(=O)...,MKTLLLDLALWSLLFQPGWLSFSSQVSQNCHNGSYEISVLMMGNSA...,1


In [ ]:
def make_set_by_values(key1, key2) :
    result = set()
    for i in range(0,len(key1)) :
        result.add((key1[i], key2[i]))
    return result
rel_true = make_set_by_values(DTI_table_2["Target_ID"], DTI_table_2["Drug_ID"])


In [35]:
import numpy as np


# generate negative samples as much as positive sample number 
# Be sure of your rel_true array, And add negative relation on rel_true array when you generate it to avoid duplicates by any chance 

neg_gene = []
neg_drug = []
count = 0

target_set = seq["Target_ID"]
drug_set = smiles.drop_duplicates("Drug_ID")["Drug_ID"]


while (True) :
    if len(target_set) == 0 :
        target_set = seq["Target_ID"]
    
    gene = np.random.choice(target_set)
    target_set = target_set[target_set != gene]
    
    drug = np.random.choice(drug_set)
    
    if (gene, drug) not in rel_true :
        neg_gene.append(gene)
        neg_drug.append(drug)
        rel_true.add((gene,drug))
        count +=1
    if count >= 9864 :
        break

In [45]:
len(neg_drug)

9864

In [37]:
neg = pd.DataFrame({"Target_ID" : neg_gene, "Drug_ID" : neg_drug})
neg_seq = pd.merge(neg, seq, on = "Target_ID")
neg_smile = pd.merge(neg_seq, smiles, on = "Drug_ID").drop_duplicates()
neg_smile["rel"] = 0

In [145]:
# build Dataframe which contains Positive sample & negative samples
df = pd.concat([neg_smile, DTI_table_2], axis = 0)

In [146]:
df

,Target_ID,Drug_ID,Target,Drug,rel
0,O94788,DB00716,MTSSKIEMPGEVKADPAALMASLHLLPSPTPNLEIKYTKIFINNEW...,CCCC1=C2C(=CC3=C1OC(=CC3=O)C(=O)O)C(=O)C=C(N2C...,0
5,P48637,DB00716,MATNWGSLLQDKQQLEELARQAVDRALAEGVLLRTSQEPTSSEVVS...,CCCC1=C2C(=CC3=C1OC(=CC3=O)C(=O)O)C(=O)C=C(N2C...,0
10,P12821,DB00716,MGAASGRRGPGLLLPLPLLLLLPPQPALALDPGLQPGNFSADEAGA...,CCCC1=C2C(=CC3=C1OC(=CC3=O)C(=O)O)C(=O)C=C(N2C...,0
15,P43155,DB00716,MKASSRFKAHQDALPRLPVPPLQQSLDHYLKALQPIVSEEEWAHTK...,CCCC1=C2C(=CC3=C1OC(=CC3=O)C(=O)O)C(=O)C=C(N2C...,0
20,P35916,DB00716,MQRGAALCLRLWLCLGLLDGLVSGYSMTPPTLNITEESHVIDTGDS...,CCCC1=C2C(=CC3=C1OC(=CC3=O)C(=O)O)C(=O)C=C(N2C...,0
...,...,...,...,...,...
9859,P25815,DB01003,MTELETAMGMIIDVFSRYSGSEGSTQTLTKGELKVLMEKELPGFLQ...,C1=CC2=C(C(=C1)OCC(COC3=CC=CC4=C3C(=O)C=C(O4)C...,1
9860,P09110,DB09069,MQRLQVVLGHLRGPADSGWMPQAAPCLSGAPQASAADVVVVHGRRT...,COC1=C(C(=C(C=C1)CN2CCNCC2)OC)OC,1
9861,P12314,DB00992,MWFLTTLLLWVPVDGQVDTTKAVITLQPPWVSVFQEETVTLHCEVL...,COC(=O)CCC(=O)CN,1
9862,P25092,DB08890,MKTLLLDLALWSLLFQPGWLSFSSQVSQNCHNGSYEISVLMMGNSA...,C[C@H]1C(=O)N[C@H]2CSSC[C@H]3C(=O)N[C@H](C(=O)...,1


In [78]:
import json

#load json file for extracting encode number of DrugId
json_file = 'node2id.json' 
with open(json_file) as f:
    data = json.load(f)

    
# Make Encode Number array    
temp_list2 = [] 
for i in df["Drug_ID"].copy() :
    exist = False
    count = 0
    for name, number in data.items() :
        length = len(data)
        if i == name :
            exist = True
            temp_list2.append(number)
            break
        elif count >= length -1 :
            temp_list2.append(np.nan)
        count += 1
len(temp_list2)

19728

In [79]:
import pandas as pd
import numpy as np
import json

#load json file for extracting encode number of TargetID
with open('entity_drug.json', 'r') as f:
    data = json.load(f)

external = pd.json_normalize(data)

external = external.T

In [96]:
temp_list = []
for i in df['NCBI'].copy() :
    exist = False
    for cn, j in enumerate(external.index) :
        length = len(external.index)
        if i == j :
            exist = True
            val = int(external[0].loc[j])
            temp_list.append(val)
            break
        elif cn == length -1 :
            temp_list.append(np.nan)
            
len(temp_list)

19728

In [84]:
# load Protein ID to Gene ID conver table
protein2gene = pd.read_csv("convert_table", delimiter="\t")

protein2gene

,UniProtKB Gene Name ID,Gene name,NCBI gene (formerly Entrezgene) ID
0,A0A023T6R1,MAGOHB,Gene::55110
1,A0A023T787,RBM8A,Gene::9939
2,A0A024QY88,NRG1,Gene::3084
3,A0A024QYR8,TM9SF2,Gene::9375
4,A0A024QYT5,SERPINE1,Gene::5054
...,...,...,...
73808,X6RLV5,DDX5,Gene::1655
73809,X6RLX0,ERC1,Gene::23085
73810,X6RLY7,CACNA2D4,Gene::93589
73811,X6RM00,ERC1,Gene::23085


In [87]:
def pro_to_gene(x) :
    
    result = protein2gene[protein2gene['UniProtKB Gene Name ID'] == x]['NCBI gene (formerly Entrezgene) ID'].values
    
    if len(result) >= 1 :
        return result[0]
    else :
        return 1

dti = pd.DataFrame()

dti['Target'] = df['Target_ID'].apply(pro_to_gene)

In [91]:
NCBI = np.array(dti)

In [95]:
df['NCBI'] = NCBI

In [97]:
df["gene_enco"] = temp_list
df["drug_enco"] = temp_list2
df

,Target_ID,Drug_ID,Target,Drug,rel,gene_enco,drug_enco,NCBI
0,O94788,DB00716,MTSSKIEMPGEVKADPAALMASLHLLPSPTPNLEIKYTKIFINNEW...,CCCC1=C2C(=CC3=C1OC(=CC3=O)C(=O)O)C(=O)C=C(N2C...,0,12009.0,NaN,Gene::8854
5,P48637,DB00716,MATNWGSLLQDKQQLEELARQAVDRALAEGVLLRTSQEPTSSEVVS...,CCCC1=C2C(=CC3=C1OC(=CC3=O)C(=O)O)C(=O)C=C(N2C...,0,10235.0,NaN,Gene::2937
10,P12821,DB00716,MGAASGRRGPGLLLPLPLLLLLPPQPALALDPGLQPGNFSADEAGA...,CCCC1=C2C(=CC3=C1OC(=CC3=O)C(=O)O)C(=O)C=C(N2C...,0,12991.0,NaN,Gene::1636
15,P43155,DB00716,MKASSRFKAHQDALPRLPVPPLQQSLDHYLKALQPIVSEEEWAHTK...,CCCC1=C2C(=CC3=C1OC(=CC3=O)C(=O)O)C(=O)C=C(N2C...,0,12640.0,NaN,Gene::1384
20,P35916,DB00716,MQRGAALCLRLWLCLGLLDGLVSGYSMTPPTLNITEESHVIDTGDS...,CCCC1=C2C(=CC3=C1OC(=CC3=O)C(=O)O)C(=O)C=C(N2C...,0,9786.0,NaN,Gene::2324
...,...,...,...,...,...,...,...,...
9859,P25815,DB01003,MTELETAMGMIIDVFSRYSGSEGSTQTLTKGELKVLMEKELPGFLQ...,C1=CC2=C(C(=C1)OCC(COC3=CC=CC4=C3C(=O)C=C(O4)C...,1,3506.0,NaN,Gene::6286
9860,P09110,DB09069,MQRLQVVLGHLRGPADSGWMPQAAPCLSGAPQASAADVVVVHGRRT...,COC1=C(C(=C(C=C1)CN2CCNCC2)OC)OC,1,13982.0,1498.0,Gene::30
9861,P12314,DB00992,MWFLTTLLLWVPVDGQVDTTKAVITLQPPWVSVFQEETVTLHCEVL...,COC(=O)CCC(=O)CN,1,12693.0,1249.0,Gene::2209
9862,P25092,DB08890,MKTLLLDLALWSLLFQPGWLSFSSQVSQNCHNGSYEISVLMMGNSA...,C[C@H]1C(=O)N[C@H]2CSSC[C@H]3C(=O)N[C@H](C(=O)...,1,12398.0,NaN,Gene::2984


In [125]:
# build Dataframe which contains full information to make Train Test Validation
all_df = df.dropna()
all_df[all_df["rel"] == 0]


,Target_ID,Drug_ID,Target,Drug,rel,gene_enco,drug_enco,NCBI
35,O94788,DB06636,MTSSKIEMPGEVKADPAALMASLHLLPSPTPNLEIKYTKIFINNEW...,C[C@@H](C1=NC(=CS1)C2=CC=C(C=C2)C#N)[C@](CN3C=...,0,12009.0,1077.0,Gene::8854
37,P18545,DB06636,MNLEPPKAEFRSATRVAGGPVTPRKGPPKFKQRQTRQFKSKPPKKG...,C[C@@H](C1=NC(=CS1)C2=CC=C(C=C2)C#N)[C@](CN3C=...,0,13943.0,1077.0,Gene::5148
39,Q9NVH6,DB06636,MWYHRLSHLHSRLQDLLKGGVIYPALPQPNFKSLLPLAVHWHHTAS...,C[C@@H](C1=NC(=CS1)C2=CC=C(C=C2)C#N)[C@](CN3C=...,0,9324.0,1077.0,Gene::55217
41,Q06187,DB06636,MAAVILESIFLKRSQQKKKTSPLNFKKRLFLLTVHKLSYYEYDFER...,C[C@@H](C1=NC(=CS1)C2=CC=C(C=C2)C#N)[C@](CN3C=...,0,3956.0,1077.0,Gene::695
43,Q16790,DB06636,MAPLCPSPWLPLLIPAPAPGLTVQLLLSLLLLVPVHPQRLPRMQED...,C[C@@H](C1=NC(=CS1)C2=CC=C(C=C2)C#N)[C@](CN3C=...,0,3606.0,1077.0,Gene::768
...,...,...,...,...,...,...,...,...
64469,Q02643,DB00889,MDRRMWGAHVFCVLSPLPTVLGHMHPECDFITQLREDESACLQAAE...,CN1[C@@H]2CCC[C@H]1CC(C2)NC(=O)C3=NN(C4=CC=CC=...,0,13915.0,383.0,Gene::2692
64474,Q8IVL6,DB00956,MLRLLRPLLLLLLLPPPGSPEPPGLTQLSPGAPPQAPDLLYADGLR...,CN1CC[C@]23[C@@H]4[C@H]1CC5=C2C(=C(C=C5)OC)O[C...,0,16808.0,1083.0,Gene::10536
64478,P20132,DB01202,MMSGEPLHVKTPIRDSMALSKMAGTSVYLKMDSAQPSGSFKIRGIG...,CC[C@@H](C(=O)N)N1CCCC1=O,0,18862.0,1271.0,Gene::10993
64482,P16662,DB01202,MSVKWTSVILLIQLSFCFSSGNCGKVLVWAAEYSHWMNIKTILDEL...,CC[C@@H](C(=O)N)N1CCCC1=O,0,16095.0,1271.0,Gene::7364


165

In [113]:
import numpy as np


# repeat to make negative samples to equalize the number of pos, neg samples
neg_gene_p = []
neg_drug_p = []
count = 0

target_set = all_df.drop_duplicates("Target_ID")["Target_ID"]
drug_set = all_df.drop_duplicates("Drug_ID")["Drug_ID"]


while (True) :
    if len(target_set) == 0 :
        target_set = all_df["Target_ID"]
    
    gene = np.random.choice(target_set)
    target_set = target_set[target_set != gene]
    
    drug = np.random.choice(drug_set)
    
    if (gene, drug) not in rel_true :
        neg_gene_p.append(gene)
        neg_drug_p.append(drug)
        rel_true.add((gene,drug))
        count +=1
    if count >= 165 :
        break


['P24298',
 'P48651',
 'P10606',
 'P49888',
 'P21964',
 'P11166',
 'O60568',
 'Q05932',
 'P10620',
 'Q7Z2Z2',
 'P08253',
 'Q9NPA1',
 'Q9NS40',
 'P28845',
 'Q86W47',
 'P30531',
 'P24666',
 'P06732',
 'P08684',
 'Q9Y2Q3',
 'P23921',
 'P00367',
 'Q9HAB3',
 'Q5T3U5',
 'P10828',
 'P41595',
 'P25100',
 'Q8N119',
 'P30043',
 'P04731',
 'O75306',
 'P51170',
 'Q7RTV2',
 'P08913',
 'P07202',
 'Q9BXA5',
 'P05091',
 'Q99758',
 'Q99835',
 'P11766',
 'Q86VB7',
 'P61769',
 'Q15759',
 'Q9UGM1',
 'O75795',
 'P07451',
 'P80511',
 'O60942',
 'Q9NS18',
 'Q13002',
 'P15056',
 'P11309',
 'P35346',
 'Q06187',
 'O00748',
 'Q8N8R3',
 'P07237',
 'Q96EY8',
 'P43155',
 'Q13838',
 'P78329',
 'P41252',
 'Q96DP5',
 'A8MPY1',
 'P33121',
 'Q9Y6M7',
 'P00505',
 'P35916',
 'Q96NT5',
 'Q969P6',
 'O43741',
 'O60733',
 'P12532',
 'P33261',
 'P49619',
 'P35626',
 'P48544',
 'P18440',
 'O15244',
 'O94760',
 'Q96RP8',
 'P28482',
 'Q8NET8',
 'P15104',
 'Q14181',
 'P13686',
 'O95831',
 'Q9NVH6',
 'Q96LZ3',
 'Q99456',
 'Q9ULW8',

In [115]:
neg_p = pd.DataFrame({"Target_ID" : neg_gene_p, "Drug_ID" : neg_drug_p})
neg_seq_p = pd.merge(neg_p, seq, on = "Target_ID")
neg_smile_p = pd.merge(neg_seq_p, smiles, on = "Drug_ID").drop_duplicates()
neg_smile_p["rel"] = 0
neg_smile_p

,Target_ID,Drug_ID,Target,Drug,rel
0,P24298,DB00607,MASSTGDRSQAVRHGLRAKVLTLDGMNPRVRRVEYAVRGPIVQRAL...,CCOC1=C(C2=CC=CC=C2C=C1)C(=O)N[C@H]3[C@@H]4N(C...,0
3,P48651,DB00275,MASCVGSRTLSKDDVNYKMHFRMINEQQVEDITIDFFYRPHTITLL...,CCCC1=NC(=C(N1CC2=CC=C(C=C2)C3=CC=CC=C3C4=NNN=...,0
7,P10606,DB00800,MASRLLRGAGTLAAQALRARGPSGAAAMRSMASGGGVPTDEEQATG...,C1CNCC(C2=CC(=C(C(=C21)Cl)O)O)C3=CC=C(C=C3)O,0
15,P49888,DB04920,MNSELDYYEKFEEVHGILMYKDFVKYWDNVEAFQARPDDLVIATYP...,CCCC(=O)OCOC(=O)C1=C(NC(=C(C1C2=C(C(=CC=C2)Cl)...,0
26,P21964,DB08819,MGDTKEQRILNHVLQHAEPGNAQSVLEAIDTYCEQKEWAMNVGDKK...,CC(C)OC(=O)CCC/C=C\C[C@H]1[C@H](C[C@H]([C@@H]1...,0
...,...,...,...,...,...
1067,Q9H244,DB00449,MQAVDNLTSAPGNTSLCTRDYKITQVLFPLLYTVLFFVGLITNGLA...,CC(C)(C)C(=O)OC1=C(C=C(C=C1)C(CNC)O)OC(=O)C(C)...,0
1072,P12081,DB01183,MAERAALEELVKLQGERVRGLKQQKASAELIEEEVAKLLKLKAQLG...,C=CCN1CC[C@]23[C@@H]4C(=O)CC[C@]2([C@H]1CC5=C3...,0
1083,Q8TDS4,DB00457,MNRHHLQDHFLEIDKKNCCVFRDDFIVKVLPPVLGLEFIFGLLGNG...,COC1=C(C=C2C(=C1)C(=NC(=N2)N3CCN(CC3)C(=O)C4=C...,0
1098,O14732,DB00850,MKPSGQDRKALTEEKRVSTKTSAADLVTETDHLVEDLIISELRERF...,C1CN(CCN1CCCN2C3=CC=CC=C3SC4=C2C=C(C=C4)Cl)CCO,0


In [118]:
import json

# JSON 파일 경로 설정
json_file = 'node2id.json'

# JSON 파일 읽기
with open(json_file) as f:
    data = json.load(f)

temp_list2 = [] 
for i in neg_smile_p["Drug_ID"].copy() :
    exist = False
    count = 0
    for name, number in data.items() :
        length = len(data)
        if i == name :
            exist = True
            temp_list2.append(number)
            break
        elif count >= length -1 :
            temp_list2.append(np.nan)
        count += 1
len(temp_list2)

import pandas as pd
import numpy as np
import json

with open('entity_drug.json', 'r') as f:
    data = json.load(f)

external = pd.json_normalize(data)

external = external.T

def pro_to_gene(x) :
    
    result = protein2gene[protein2gene['UniProtKB Gene Name ID'] == x]['NCBI gene (formerly Entrezgene) ID'].values
    
    if len(result) >= 1 :
        return result[0]
    else :
        return 1

dti = pd.DataFrame()

dti['Target'] = neg_smile_p['Target_ID'].apply(pro_to_gene)

NCBI = np.array(dti)

neg_smile_p['NCBI'] = NCBI

temp_list = []
for i in neg_smile_p['NCBI'].copy() :
    exist = False
    for cn, j in enumerate(external.index) :
        length = len(external.index)
        if i == j :
            exist = True
            val = int(external[0].loc[j])
            temp_list.append(val)
            break
        elif cn == length -1 :
            temp_list.append(np.nan)
            
len(temp_list)


neg_smile_p["gene_enco"] = temp_list
neg_smile_p["drug_enco"] = temp_list2
neg_smile_p

,Target_ID,Drug_ID,Target,Drug,rel,NCBI,gene_enco,drug_enco
0,P24298,DB00607,MASSTGDRSQAVRHGLRAKVLTLDGMNPRVRRVEYAVRGPIVQRAL...,CCOC1=C(C2=CC=CC=C2C=C1)C(=O)N[C@H]3[C@@H]4N(C...,0,Gene::2875,14664,157
3,P48651,DB00275,MASCVGSRTLSKDDVNYKMHFRMINEQQVEDITIDFFYRPHTITLL...,CCCC1=NC(=C(N1CC2=CC=C(C=C2)C3=CC=CC=C3C4=NNN=...,0,Gene::9791,14498,1054
7,P10606,DB00800,MASRLLRGAGTLAAQALRARGPSGAAAMRSMASGGGVPTDEEQATG...,C1CNCC(C2=CC(=C(C(=C21)Cl)O)O)C3=CC=C(C=C3)O,0,Gene::1329,5927,1058
15,P49888,DB04920,MNSELDYYEKFEEVHGILMYKDFVKYWDNVEAFQARPDDLVIATYP...,CCCC(=O)OCOC(=O)C1=C(NC(=C(C1C2=C(C(=CC=C2)Cl)...,0,Gene::6783,2303,662
26,P21964,DB08819,MGDTKEQRILNHVLQHAEPGNAQSVLEAIDTYCEQKEWAMNVGDKK...,CC(C)OC(=O)CCC/C=C\C[C@H]1[C@H](C[C@H]([C@@H]1...,0,Gene::1312,9608,1578
...,...,...,...,...,...,...,...,...
1067,Q9H244,DB00449,MQAVDNLTSAPGNTSLCTRDYKITQVLFPLLYTVLFFVGLITNGLA...,CC(C)(C)C(=O)OC1=C(C=C(C=C1)C(CNC)O)OC(=O)C(C)...,0,Gene::64805,12837,704
1072,P12081,DB01183,MAERAALEELVKLQGERVRGLKQQKASAELIEEEVAKLLKLKAQLG...,C=CCN1CC[C@]23[C@@H]4C(=O)CC[C@]2([C@H]1CC5=C3...,0,Gene::3035,2552,261
1083,Q8TDS4,DB00457,MNRHHLQDHFLEIDKKNCCVFRDDFIVKVLPPVLGLEFIFGLLGNG...,COC1=C(C=C2C(=C1)C(=NC(=N2)N3CCN(CC3)C(=O)C4=C...,0,Gene::338442,18871,703
1098,O14732,DB00850,MKPSGQDRKALTEEKRVSTKTSAADLVTETDHLVEDLIISELRERF...,C1CN(CCN1CCCN2C3=CC=CC=C3SC4=C2C=C(C=C4)Cl)CCO,0,Gene::3613,6583,525


In [127]:
#all_df = pd.concat([neg_smile_p, all_df], axis = 0)
#all_df = all_df.dropna()
all_df[all_df["rel"] == 1]


,Target_ID,Drug_ID,Target,Drug,rel,NCBI,gene_enco,drug_enco
0,Q16348,DB00672,MNPFQKNESKETLFSPVSIEEVPPRPPSPPKKPSPTICGSNYPLSI...,CCCNC(=O)NS(=O)(=O)C1=CC=C(C=C1)Cl,1,Gene::6565,15094.0,569.0
1,Q16348,DB00577,MNPFQKNESKETLFSPVSIEEVPPRPPSPPKKPSPTICGSNYPLSI...,CC(C)[C@@H](C(=O)OCCOCN1C=NC2=C1N=C(NC2=O)N)N,1,Gene::6565,15094.0,1275.0
2,Q16348,DB01212,MNPFQKNESKETLFSPVSIEEVPPRPPSPPKKPSPTICGSNYPLSI...,CN1C(=NC(=O)C(=O)N1)SCC2=C(N3[C@@H]([C@@H](C3=...,1,Gene::6565,15094.0,869.0
3,Q16348,DB01112,MNPFQKNESKETLFSPVSIEEVPPRPPSPPKKPSPTICGSNYPLSI...,CO/N=C(/C1=CC=CO1)\C(=O)N[C@H]2[C@@H]3N(C2=O)C...,1,Gene::6565,15094.0,748.0
4,Q16348,DB00456,MNPFQKNESKETLFSPVSIEEVPPRPPSPPKKPSPTICGSNYPLSI...,CC(=O)OCC1=C(N2[C@@H]([C@@H](C2=O)NC(=O)CC3=CC...,1,Gene::6565,15094.0,871.0
...,...,...,...,...,...,...,...,...
9857,O43613,DB09034,MEPSATPGAQMGVPPGSREPSPVPPDYEDEFLRYLWRDYLYPKQYE...,C[C@@H]1CCN(CCN1C(=O)C2=C(C=CC(=C2)C)N3N=CC=N3...,1,Gene::3061,14986.0,1258.0
9858,O43614,DB09034,MSGTKLEDSPPCRNWSSASELNETQEPFLNPTDYDDEEFLRYLWRE...,C[C@@H]1CCN(CCN1C(=O)C2=C(C=CC(=C2)C)N3N=CC=N3...,1,Gene::3062,8451.0,1258.0
9860,P09110,DB09069,MQRLQVVLGHLRGPADSGWMPQAAPCLSGAPQASAADVVVVHGRRT...,COC1=C(C(=C(C=C1)CN2CCNCC2)OC)OC,1,Gene::30,13982.0,1498.0
9861,P12314,DB00992,MWFLTTLLLWVPVDGQVDTTKAVITLQPPWVSVFQEETVTLHCEVL...,COC(=O)CCC(=O)CN,1,Gene::2209,12693.0,1249.0


In [131]:
import pandas as pd
import numpy as np

# Load the dataset as a pandas DataFrame

# Shuffle the DataFrame rows
all_df = all_df.sample(frac=1, random_state=42).reset_index(drop=True).dropna()
all_df.columns = ["Target_ID", "Drug_ID", "Target", "Drug", "Y", "NCBI_ID", "gene_enco", "drug_enco"]
# Split the dataset into training, testing, and validation sets
train_size = 0.7
test_size = 0.2
val_size = 0.1

train_split_index = int(len(df) * train_size)
test_split_index = int(len(df) * (train_size + test_size))

train_data = all_df.iloc[:train_split_index]
test_data = all_df.iloc[train_split_index:test_split_index]
val_data = all_df.iloc[test_split_index:]

# Print the shapes of the resulting datasets
print('Training data shape:', train_data.shape)
print('Testing data shape:', test_data.shape)
print('Validation data shape:', val_data.shape)

train_data.to_csv('vec_train_origin.csv',index=False)
test_data.to_csv('vec_test_origin.csv',index=False)
val_data.to_csv('vec_dev_origin.csv', index=False)

Training data shape: (12331, 8)
Testing data shape: (3524, 8)
Validation data shape: (1927, 8)


In [66]:
df["Target_ID"]

0       O94788
5       P48637
10      P12821
15      P43155
20      P35916
         ...  
9859    P25815
9860    P09110
9861    P12314
9862    P25092
9863    P39023
Name: Target_ID, Length: 19728, dtype: object

In [138]:
import pandas as pd

# Save DataFrame to run DTi2Vec
all_df[all_df["Y"]==0][["Target_ID", "Drug_ID"]].to_csv('R_DrugBank_neg.txt', index=False, header=None, sep='\t')

In [134]:
pd.read_csv("vec_dev_origin.csv")[["Target_ID", "Drug_ID"]].to_csv('dev_sample_file.txt', index=False, header=None, sep='\t')